# Aggregate the mms data into one point observation
# the several points of observations per day into daily records 

posible approaches is to create aggreate the mms into point observation
Aggregate these point observations into daily records - may be to weight them? 

In [5]:
import pandas as pd
import datetime

In [40]:
#I have created a file with some more observations in order to be able to test aggregation capabilities
data = pd.read_csv("/Users/rmania/repos/Kaufland Case/sample_Kaufland_case_IoT_and_predictive_maintenance.csv",\
                   sep = ';', parse_dates = [2,3,4])

In [41]:
data.head()

,machine_name,sensor_type,date_measurement,start_timestamp,end_timestamp,realvalue,unit
0,RBG1,drive_gear_V_eff,2016-02-09,2016-02-09 15:26:00,2016-02-09 15:26:00,0.395,mm/s
1,RBG1,drive_gear_V_eff,2016-02-09,2016-02-09 15:26:00,2016-02-09 15:26:00,0.577,mm/s
2,RBG1,drive_gear_V_eff,2016-02-09,2016-02-09 15:26:00,2016-02-09 15:26:00,0.717,mm/s
3,RBG1,drive_gear_V_eff,2016-02-09,2016-02-09 15:26:00,2016-02-09 15:26:00,0.832,mm/s
4,RBG1,drive_gear_V_eff,2016-02-09,2016-02-09 15:26:00,2016-02-09 15:26:00,0.941,mm/s


In [122]:
#Aggregate on daily basis - min, max, mean, spread, etc.
daily_aggr = pd.DataFrame({'Daily_min': data['realvalue'].groupby(data['date_measurement']).min(), \
                           'Daily_max': data['realvalue'].groupby(data['date_measurement']).max(), \
                           'Daily_mean': data['realvalue'].groupby(data['date_measurement']).mean(), \
                           'Daily_spread': data['realvalue'].groupby(data['date_measurement']).max() - \
                           data['realvalue'].groupby(data['date_measurement']).min()})

In [123]:
daily_aggr

,Daily_max,Daily_mean,Daily_min,Daily_spread
date_measurement,,,,
2016-02-09,1.62,1.097357,0.395,1.225
2016-03-09,1.62,1.097357,0.395,1.225
2016-04-09,1.62,1.053026,0.395,1.225
2016-05-09,1.62,1.097357,0.395,1.225
2016-07-09,1.62,1.097357,0.395,1.225
2016-08-09,1.62,1.097357,0.395,1.225


In [83]:
#Label the morning, afternoon and evening measurements to be able to discriminate, aggregate, etc
data['Measurement_index'] = pd.Series([])

for i in range(data.shape[0]):
    if (data['start_timestamp'][i].time() > datetime.time(9, 00)) and \
         (data['start_timestamp'][i].time() < datetime.time(15, 00)):
        data['Measurement_index'][i] = 'Morning'
    elif (data['start_timestamp'][i].time() > datetime.time(15, 00)) and \
         (data['start_timestamp'][i].time() < datetime.time(18, 00)):
        data['Measurement_index'][i] = 'Afternoon'
    else:
        data['Measurement_index'][i] = 'Evening'

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWith

In [84]:
data['Measurement_index'].value_counts()

Morning      70
Afternoon    60
Evening      48
Name: Measurement_index, dtype: int64

In [85]:
data.groupby(['date_measurement','Measurement_index'])['realvalue'].min()

date_measurement  Measurement_index
2016-02-09        Afternoon            0.395
                  Evening              1.417
                  Morning              0.395
2016-03-09        Afternoon            0.395
                  Evening              1.417
                  Morning              0.395
2016-04-09        Afternoon            0.395
                  Evening              1.417
                  Morning              0.395
2016-05-09        Afternoon            0.395
                  Evening              1.417
                  Morning              0.395
2016-07-09        Afternoon            0.395
                  Evening              1.417
                  Morning              0.395
2016-08-09        Afternoon            0.395
                  Evening              1.417
                  Morning              0.395
Name: realvalue, dtype: float64

In [110]:
#Aggregate on daily and hourly basis
daily_hourly_aggr = pd.DataFrame({'Daily_hourly_min': data.groupby(['date_measurement','Measurement_index'])['realvalue'].min(), \
                           'Daily_hourly_max': data.groupby(['date_measurement','Measurement_index'])['realvalue'].max(), \
                           'Daily_hourly_mean': data.groupby(['date_measurement','Measurement_index'])['realvalue'].mean(), \
                           'Daily_hourly_spread': data.groupby(['date_measurement','Measurement_index'])['realvalue'].max() - \
                           data.groupby(['date_measurement','Measurement_index'])['realvalue'].min(), \
                                'Daily_hourly_max_pct_change': data.groupby(['date_measurement','Measurement_index'])['realvalue'].pct_change().max(), \
                                 'Daily_hourly_min_pct_change': data.groupby(['date_measurement','Measurement_index'])['realvalue'].pct_change().min()}).reset_index()

In [109]:
daily_hourly_aggr.head()

,date_measurement,Measurement_index,Daily_hourly_max,Daily_hourly_max_pct_change,Daily_hourly_mean,Daily_hourly_min,Daily_hourly_spread
0,2016-02-09,Afternoon,1.258,0.460759,0.9289,0.395,0.863
1,2016-02-09,Evening,1.620,0.460759,1.5185,1.417,0.203
2,2016-02-09,Morning,1.258,0.460759,0.9289,0.395,0.863
3,2016-03-09,Afternoon,1.258,0.460759,0.9289,0.395,0.863
4,2016-03-09,Evening,1.620,0.460759,1.5185,1.417,0.203


In [111]:
#It is easy to split out only the morning measurements
daily_hourly_aggr[daily_hourly_aggr['Measurement_index'] == 'Morning']

,date_measurement,Measurement_index,Daily_hourly_max,Daily_hourly_max_pct_change,Daily_hourly_mean,Daily_hourly_min,Daily_hourly_min_pct_change,Daily_hourly_spread
2,2016-02-09,Morning,1.258,0.460759,0.9289,0.395,-0.68601,0.863
5,2016-03-09,Morning,1.258,0.460759,0.9289,0.395,-0.68601,0.863
8,2016-04-09,Morning,1.258,0.460759,0.9289,0.395,-0.68601,0.863
11,2016-05-09,Morning,1.258,0.460759,0.9289,0.395,-0.68601,0.863
14,2016-07-09,Morning,1.258,0.460759,0.9289,0.395,-0.68601,0.863
17,2016-08-09,Morning,1.258,0.460759,0.9289,0.395,-0.68601,0.863


In [105]:
daily_hourly_aggr[daily_hourly_aggr['Measurement_index'] == 'Afternoon']

,date_measurement,Measurement_index,Daily_hourly_max,Daily_hourly_mean,Daily_hourly_min,Daily_hourly_spread
0,2016-02-09,Afternoon,1.258,0.9289,0.395,0.863
3,2016-03-09,Afternoon,1.258,0.9289,0.395,0.863
6,2016-04-09,Afternoon,1.258,0.9289,0.395,0.863
9,2016-05-09,Afternoon,1.258,0.9289,0.395,0.863
12,2016-07-09,Afternoon,1.258,0.9289,0.395,0.863
15,2016-08-09,Afternoon,1.258,0.9289,0.395,0.863


In [126]:
#Find the difference in days between consequent observations
#Obviously I have imported the dates incorrectly
for i in range(daily_aggr.shape[0]-1):
    print((daily_aggr.index[i+1] - daily_aggr.index[i]).days)

29
31
30
61
31
